In [ ]:
import pandas as pd
import numpy as np
import newsdataapi
import json
import matplotlib.pyplot as plt
import re
import os
from dotenv import load_dotenv
from openai import OpenAI
import requests
from io import StringIO
import bcchapi
import openpyxl

load_dotenv()
client = OpenAI(api_key = os.getenv('OPEN_API_KEY'))

In [ ]:
response = client.responses.create(
    model="gpt-4o",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)


# OCDE FUNCTION

In [ ]:
def get_csv_from_api_OCDE(url):
    try:
        # Make the request
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        
        # Convert response content to DataFrame
        # Use StringIO to create a file-like object from the response text
        from io import StringIO
        df = pd.read_csv(StringIO(response.text))
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error making API request: {e}")
        return None
    except Exception as e:
        print(f"Error processing CSV data: {e}")
        return None

# UNEMPLOYMENT

In [ ]:
url = 'https://sdmx.oecd.org/public/rest/data/OECD.SDD.TPS,DSD_LFS@DF_IALFS_UNE_M,1.0/CHL+MEX+COL+CRI..._Z.Y._T.Y_GE15..M?startPeriod=2024-02&format=csvfilewithlabels'

df_unemployment = get_csv_from_api_OCDE(url)

# MEXICO

In [ ]:
token_mex = os.getenv('API_MEX')
def get_df_from_inegi(indicator_id, token = token_mex):
    url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{indicator_id}/es/0700/false/BIE/2.0/{token}"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data["Series"][0]['OBSERVATIONS'])[["TIME_PERIOD","OBS_VALUE"]]
    return df

In [ ]:
#indicador_id = '910406' #inflation

# First request for GDP proxy (737220)
df_gdp_proxy = get_df_from_inegi("737220")

# Second request for unemployment (444884)
df_unemployment = get_df_from_inegi("444884")

# Inflation
df_inflation_mex = pd.read_excel('Data/df_infl_mex.xlsx', skiprows=4)
df_inflation_mex = df_inflation_mex.rename(columns={'Periodos': 'Date', 'Precios e Inflación': 'Inflation'})
df_inflation_mex.columns = ['date', 'Inflation']

# Merge the dataframes on the date column
df_gdp_proxy.columns = ['date', 'IGAE']
df_unemployment.columns = ['date', 'unemployment']
df_mexico = pd.merge(df_gdp_proxy, df_unemployment, on='date', how='outer')

# Final merge
df_mexico = df_mexico.sort_values('date', ascending=False)
df_mexico = pd.merge(df_mexico, df_inflation_mex, on='date', how='outer')


# CHILE

In [ ]:
api_chile = bcchapi.Siete(os.getenv('CORREO_CHILE'), os.getenv('PASSWORD_CHILE'))

df_chile = api_chile.cuadro(
    series = ["F032.IMC.IND.Z.Z.EP18.Z.Z.0.M", "F074.IPC.V12.Z.EP23.C.M", "F049.DES.TAS.INE9.10.M"],
    nombres = ["imacec", "ipc", "unemployment"],
    desde = "2024-01-01",
    hasta = "2025-03-31",
    frecuencia = "ME",
    observado = {"imacec":"last", "ipc":"last", "unemployment":"last"}
)   

# COSTA RICA

In [ ]:
token_cr = os.getenv('API_CR')
def get_df_from_cr(indicator_id, start_period, end_period, sub_level, indicator_name,
                   name = 'Ahias', token = token_cr, email = os.getenv('CORREO_CHILE')):
    
    url = 'https://gee.bccr.fi.cr/Indicadores/Suscripciones/WS/wsindicadoreseconomicos.asmx/ObtenerIndicadoresEconomicosXML'
    url = url + f'?Indicador={indicator_id}&FechaInicio={start_period}&FechaFinal={end_period}&Nombre={name}&SubNiveles={sub_level}'
    url = url + f'&CorreoElectronico={email}&Token={os.getenv("API_CR")}'
    response = requests.get(url)
    
    clean_text = response.text.replace('<?xml version="1.0" encoding="utf-8"?>', '')
    clean_text = re.sub(r'<[^>]+>', '', clean_text)
    clean_text = re.sub(r'(&lt;)|(&gt)', '', clean_text)
    clean_text = re.sub(r"/[A-Z]+_[A-Z]+", '', clean_text)
    clean_text = re.sub(r"(Datos_de_INGC011_CAT_INDICADORECONOMIC;)|(INGC011_CAT_INDICADORECONOMIC;)", '', clean_text)
    clean_text = re.sub(r'/', '', clean_text)
    clean_text = ([line.strip() for line in clean_text.split('\n') if line.strip()])
    
    dates = np.array([])
    values = np.array([])

    for element in clean_text:
        divided = re.split(r";", element)
        if divided[0] == 'DES_FECHA':
            date_str = divided[1].split('T')[0]
            dates = np.append(dates, pd.to_datetime(date_str))
        if divided[0] == 'NUM_VALOR':      
            values = np.append(values, float(divided[1]))
            
    df = pd.DataFrame({'date': dates,
                       indicator_name: values})
    return df

In [ ]:
df_cr_gfp_proxy = get_df_from_cr(87764, '01/01/2025', '18/04/2025', 'N', 'IMAE') #gdp proxy
df_cr_unempl = get_df_from_cr(23633, '01/01/2025', '18/04/2025', 'N', 'unemloyment') #unemloyment
df_cr_inf = get_df_from_cr(89638, '01/01/2025', '18/04/2025', 'N', 'var_ipc') #inflation

df_cr = pd.merge(df_cr_gfp_proxy, df_cr_unempl, on='date', how='outer')
df_cr = pd.merge(df_cr, df_cr_inf, on = 'date', how = 'outer')
